In [6]:
!pip install pandas splink nicknames


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import pandas as pd

import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on
from nicknames import NickNamer

nn = NickNamer()

data = [
    {"unique_id": 1, "first_name": "John", "last_name": "Smith", "gender" : "male"},
    {"unique_id": 2, "first_name": "John", "last_name": "Smiths", "gender" : "male"},
    {"unique_id": 3, "first_name": "Johnny", "last_name": "Smith", "gender" : "female"},
    {"unique_id": 4, "first_name": "Jonnie", "last_name": "Wilkins", "gender" : "male"},
    {"unique_id": 5, "first_name": "Johns", "last_name": "Smith", "gender" : "female"}
]

df = pd.DataFrame(data)


In [45]:
list_first_name_nicks=[]
list_last_name_nicks=[]

for index, row in df.iterrows():
    first_name = row['first_name']
    last_name = row['last_name']
    nicks = nn.nicknames_of(first_name)
    list_first_name_nicks.append(list(nicks))
    nicks = nn.nicknames_of(last_name)
    list_last_name_nicks.append(list(nicks))

df['nicks_first_name'] = list_first_name_nicks
df['nicks_last_name'] = list_last_name_nicks

In [46]:
print(df)

   unique_id first_name last_name  gender  \
0          1       John     Smith    male   
1          2       John    Smiths    male   
2          3     Johnny     Smith  female   
3          4     Jonnie   Wilkins    male   
4          5      Johns     Smith  female   

                                nicks_first_name nicks_last_name  
0  [jack, jonny, jonnie, jon, johnny, ian, jock]        [smitty]  
1  [jack, jonny, jonnie, jon, johnny, ian, jock]              []  
2                                             []        [smitty]  
3                                             []              []  
4                                             []        [smitty]  


In [47]:
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
{
            "output_column_name": "first_name",
            "comparison_levels": [
                {
                    "sql_condition": "\"first_name_l\" IS NULL OR \"first_name_r\" IS NULL",
                    "label_for_charts": "first_name is NULL",
                    "is_null_level": True
                },
                {
                    "sql_condition": "\"first_name_l\" = \"first_name_r\"",
                    "label_for_charts": "Exact match on first_name"
                },
                {
                    "sql_condition": "jaro_winkler_similarity(\"first_name_l\", \"first_name_r\") >= 0.9",
                    "label_for_charts": "Jaro-Winkler distance of first_name >= 0.9"
                },
                {
                    "sql_condition": "jaro_winkler_similarity(\"first_name_l\", \"first_name_r\") >= 0.8",
                    "label_for_charts": "Jaro-Winkler distance of first_name >= 0.8"
                },
                {
                    "sql_condition": "list_contains(nicks_first_name_l, first_name_r) OR list_contains(nicks_first_name_r, first_name_l)",
                    "label_for_charts": "Nick Name comparision for first_name"
                },
                {
                    "sql_condition": "ELSE",
                    "label_for_charts": "All other comparisons"
                }
            ],
            "comparison_description": "nick name and jaro_winkler at thresholds 0.9, 0.8 vs. anything else"
        },        
{
            "output_column_name": "last_name",
            "comparison_levels": [
                {
                    "sql_condition": "\"last_name_l\" IS NULL OR \"last_name_r\" IS NULL",
                    "label_for_charts": "last_name is NULL",
                    "is_null_level": True
                },
                {
                    "sql_condition": "\"last_name_l\" = \"last_name_r\"",
                    "label_for_charts": "Exact match on last_name"
                },
                {
                    "sql_condition": "jaro_winkler_similarity(\"last_name_l\", \"last_name_r\") >= 0.9",
                    "label_for_charts": "Jaro-Winkler distance of last_name >= 0.9"
                },
                {
                    "sql_condition": "jaro_winkler_similarity(\"last_name_l\", \"last_name_r\") >= 0.8",
                    "label_for_charts": "Jaro-Winkler distance of last_name >= 0.8"
                },
                {
                    "sql_condition": "list_contains(nicks_last_name_l, last_name_r) OR list_contains(nicks_last_name_r, last_name_l)",
                    "label_for_charts": "Nick Name comparision for last_name"
                },
                {
                    "sql_condition": "ELSE",
                    "label_for_charts": "All other comparisons"
                }
            ],
            "comparison_description": "nick name and jaro_winkler at thresholds 0.9, 0.8 vs. anything else"
        },
        cl.ExactMatch("gender")
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        block_on("last_name"),
        block_on("gender")
    ],
    max_iterations=2,
)

In [48]:
linker = Linker(df, settings, DuckDBAPI(), set_up_basic_logging=False)
df_predict = linker.inference.predict()
df_e = df_predict.as_pandas_dataframe(limit=5)
df_e


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'first_name':
    u values not fully trained
Comparison: 'last_name':
    m values not fully trained
Comparison: 'last_name':
    u values not fully trained
Comparison: 'gender':
    m values not fully trained
Comparison: 'gender':
    u values not fully trained
The 'probability_two_random_records_match' setting has been set to the default value (0.0001). 
If this is not the desired behaviour, either: 
 - assign a value for `probability_two_random_records_match` in your settings dictionary, or 
 - estimate with the `linker.estimate_probability_two_random_records_match` function.


,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,nicks_first_name_l,nicks_first_name_r,gamma_first_name,last_name_l,last_name_r,nicks_last_name_l,nicks_last_name_r,gamma_last_name,gender_l,gender_r,gamma_gender,match_key
0,9.712432,0.998809,1,2,John,John,"[jack, jonny, jonnie, jon, johnny, ian, jock]","[jack, jonny, jonnie, jon, johnny, ian, jock]",4,Smith,Smiths,[smitty],[],3,male,male,1,0
1,-5.287568,0.024963,1,3,John,Johnny,"[jack, jonny, jonnie, jon, johnny, ian, jock]",[],3,Smith,Smith,[smitty],[smitty],4,male,female,0,1
2,-7.954235,0.004016,2,4,John,Jonnie,"[jack, jonny, jonnie, jon, johnny, ian, jock]",[],2,Smiths,Wilkins,[],[],0,male,male,1,2
3,7.045765,0.992488,3,5,Johnny,Johns,[],[],2,Smith,Smith,[smitty],[smitty],4,female,female,1,1
4,-7.954235,0.004016,1,4,John,Jonnie,"[jack, jonny, jonnie, jon, johnny, ian, jock]",[],2,Smith,Wilkins,[smitty],[],0,male,male,1,2


In [49]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    df_predict, threshold_match_probability=0.9
)
clusters.as_pandas_dataframe(limit=5)

,cluster_id,unique_id,first_name,last_name,gender,nicks_first_name,nicks_last_name
0,1,1,John,Smith,male,"[jack, jonny, jonnie, jon, johnny, ian, jock]",[smitty]
1,1,2,John,Smiths,male,"[jack, jonny, jonnie, jon, johnny, ian, jock]",[]
2,3,3,Johnny,Smith,female,[],[smitty]
3,4,4,Jonnie,Wilkins,male,[],[]
4,3,5,Johns,Smith,female,[],[smitty]
